# 승촌보(용봉) 데이터

- 영산강 승촌보(용봉) 데이터 살펴 보기
- 시차 데이터 첫 적용

**필수 라이브러리**

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

**matplotlib 한글 설정**

- 운영체제에 따른 한글 지원 설정. 윈도우, 우분투, 구글 코랩 지원.
- 참고: [matplotlib에서 한글 지원하기](https://github.com/codingalzi/datapy/blob/master/matplotlib-korean.md)

In [8]:
import platform
plt.rc('figure', figsize=(10, 6))  # 그림 크기 설정
if platform.system() == 'Windows': # 윈도우
    from matplotlib import font_manager, rc
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
elif platform.system() == 'Linux': # 우분투 또는 구글 코랩
    # !sudo apt-get install -y fonts-nanum*
    # !fc-cache -fv
    
    applyfont = "NanumBarunGothic"
    import matplotlib.font_manager as fm
    if not any(map(lambda ft: ft.name == applyfont, fm.fontManager.ttflist)):
        fm.fontManager.addfont("/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf")
    plt.rc("font", family=applyfont)
    plt.rc("axes", unicode_minus=False)

# 데이터 가져오기

In [9]:
# base_path = '/content/drive/MyDrive/water_data/'
base_path = "https://github.com/codingalzi/water-data/raw/master/reservoirs/"

## 승촌보(용봉) 자료를 데이터프레임으로 만들기  

* `skiprows=0`: 0번 행 무시. 모든 워크시트 대상.
* `header=1`: 1번 행을 header로 지정, 즉 열 인덱스로 사용.
* `sheet_name=None`: 모든 워크시트 가져오기. 워크시트별로 하나의 df 생성. 반환값은 사전.
* `na_values='ND'`: ND(No Data/Not Determinded) 또한 결측치로 처리. 용조산소 열에 포함되어 있음.
* `index_col=0`: 측정일을 행 인덱스로 사용
* `parse_dates=True`: 행 인덱스로 사용되는 날짜 대상 파싱 실행

In [6]:
scb_yearly = pd.read_excel(base_path+"Seungchonbo.xlsx", 
                            skiprows=0, 
                            header=1, 
                            sheet_name=None,
                            na_values='ND',
                            parse_dates=True)  # index_col = 0을 제외함

In [7]:
print(type(scb_yearly))  # type-dict
print(scb_yearly.keys())  # 2012년부터 2022년까지

<class 'dict'>
dict_keys(['2012년', '2013년', '2014년', '2015년', '2016년', '2017년', '2018년', '2019년', '2020년', '2021년', '2022년'])


In [8]:
print(scb_yearly['2014년'].head())

            측정일/월 측정소명  수온 ℃  수소이온농도  전기전도도 μS/cm  용존산소 mg/L  총유기탄소 mg/L  \
0  2014-01-01 00h   용봉   5.8     6.7        412.0        8.4         3.5   
1  2014-01-01 01h   용봉   5.8     6.7        412.0        8.3         3.4   
2  2014-01-01 02h   용봉   5.9     6.7        418.0        8.0         3.4   
3  2014-01-01 03h   용봉   5.9     6.7        421.0        7.8         3.5   
4  2014-01-01 04h   용봉   5.9     6.7        419.0        7.9         3.4   

   총질소 mg/L  총인 mg/L  클로로필-a mg/㎥  
0     6.419    0.058         12.9  
1     6.358    0.059         12.6  
2     6.407    0.059         11.4  
3     6.459    0.059         10.6  
4     6.486    0.059         11.1  


❗❗❗ **먼저 어레이들을 복제한 다음 행을 추가하여 기존 어레이와 병합하는 형식이 보다 빠르고 정확할 듯**

In [9]:
for year in scb_yearly:
    for var in scb_yearly[year].columns:
        scb_yearly[year][var+'(t-1)'] = 0.0
    
    for i in scb_yearly[year].index:
        for var in scb_yearly[year].columns:
            try:
                scb_yearly[year][var+'(t-1)'][i + 1] =  scb_yearly[year][var][i]
            except:
                continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


t-1 시간 데이터를 저장함. 만약, t-1 시간 데이터가 결측치라면 0.0을 넣어둠

In [10]:
print(scb_yearly['2015년'].head())

            측정일/월 측정소명  수온 ℃  수소이온농도  전기전도도 μS/cm  용존산소 mg/L  총유기탄소 mg/L  \
0  2015-01-01 00h   용봉   6.3     7.0        410.0        9.2         2.5   
1  2015-01-01 01h   용봉   6.2     7.0        410.0        9.2         2.5   
2  2015-01-01 02h   용봉   6.1     7.0        409.0        9.2         2.4   
3  2015-01-01 03h   용봉   6.1     7.0        410.0        9.2         2.4   
4  2015-01-01 04h   용봉   6.0     7.0        412.0        9.3         2.4   

   총질소 mg/L  총인 mg/L  클로로필-a mg/㎥      측정일/월(t-1) 측정소명(t-1)  수온 ℃(t-1)  \
0     5.607    0.057          3.9             0.0       0.0        0.0   
1     5.596    0.056          3.9  2015-01-01 00h        용봉        6.3   
2     5.485    0.053          3.7  2015-01-01 01h        용봉        6.2   
3     5.487    0.052          3.7  2015-01-01 02h        용봉        6.1   
4     5.625    0.052          3.8  2015-01-01 03h        용봉        6.1   

   수소이온농도(t-1)  전기전도도 μS/cm(t-1)  용존산소 mg/L(t-1)  총유기탄소 mg/L(t-1)  \
0          0.0               

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
print(scb_yearly['2015년'].isnull().sum())

측정일/월                 0
측정소명                  0
수온 ℃                361
수소이온농도              358
전기전도도 μS/cm         362
용존산소 mg/L           632
총유기탄소 mg/L          771
총질소 mg/L            516
총인 mg/L             615
클로로필-a mg/㎥         323
측정일/월(t-1)            0
측정소명(t-1)             0
수온 ℃(t-1)           361
수소이온농도(t-1)         358
전기전도도 μS/cm(t-1)    362
용존산소 mg/L(t-1)      632
총유기탄소 mg/L(t-1)     771
총질소 mg/L(t-1)       516
총인 mg/L(t-1)        615
클로로필-a mg/㎥(t-1)    323
dtype: int64


전체 결측치는 앞에 있는 값으로 대체함

In [ ]:
test_2015df = scb_yearly['2015년'].fillna(method = 'ffill')

In [ ]:
print(test_2015df.isnull().sum())

측정일/월               0
측정소명                0
수온 ℃                0
수소이온농도              0
전기전도도 μS/cm         0
용존산소 mg/L           0
총유기탄소 mg/L          0
총질소 mg/L            0
총인 mg/L             0
클로로필-a mg/㎥         0
측정일/월(t-1)          0
측정소명(t-1)           0
수온 ℃(t-1)           0
수소이온농도(t-1)         0
전기전도도 μS/cm(t-1)    0
용존산소 mg/L(t-1)      0
총유기탄소 mg/L(t-1)     0
총질소 mg/L(t-1)       0
총인 mg/L(t-1)        0
클로로필-a mg/㎥(t-1)    0
dtype: int64


In [ ]:
test_2015df.columns

Index(['측정일/월', '측정소명', '수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥', '측정일/월(t-1)',
       '측정소명(t-1)', '수온 ℃(t-1)', '수소이온농도(t-1)', '전기전도도 μS/cm(t-1)',
       '용존산소 mg/L(t-1)', '총유기탄소 mg/L(t-1)', '총질소 mg/L(t-1)', '총인 mg/L(t-1)',
       '클로로필-a mg/㎥(t-1)'],
      dtype='object')

t-1 시간 데이터와 함께 랜덤포레스트(회귀)를 돌려봄.  
예상할 수 있듯이.. 클로로필(t-1)만 중요한 변수로 확인됨. 정확도도 높은 편임.

In [ ]:
cols = ['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '수온 ℃(t-1)', '수소이온농도(t-1)', '전기전도도 μS/cm(t-1)',
       '용존산소 mg/L(t-1)', '총유기탄소 mg/L(t-1)', '총질소 mg/L(t-1)', '총인 mg/L(t-1)',
       '클로로필-a mg/㎥(t-1)']

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True)
rnd_reg.fit(test_2015df.loc[:, cols], test_2015df['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df.loc[:, cols], test_2015df['클로로필-a mg/㎥']))

0.0 수온 ℃
0.0 수소이온농도
0.0 전기전도도 μS/cm
0.0 용존산소 mg/L
0.0 총유기탄소 mg/L
0.0 총질소 mg/L
0.0 총인 mg/L
0.0 수온 ℃(t-1)
0.0 수소이온농도(t-1)
0.0 전기전도도 μS/cm(t-1)
0.0 용존산소 mg/L(t-1)
0.0 총유기탄소 mg/L(t-1)
0.0 총질소 mg/L(t-1)
0.0 총인 mg/L(t-1)
1.0 클로로필-a mg/㎥(t-1)
oob_scores : 0.9775237778701308
train data 정확도 : 0.9799746967305293


클로로필(t-1) 자료를 제외하면.. 현재 수소이온농도(0.39), 이전 수온(0.18), 현재 수온(0.15), 이전 수소이온 농도(0.1) 순서로 중요함.

클로로필이 증가해... 수소이온농도가 증가한 것이라면 중요변수로 등장한 것이 맞고, 그것을 제외하면 바로 전 시간대 수온(이전 수온)이 가장 중요한 변수가 아닐까?...

In [ ]:
cols2 = ['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '수온 ℃(t-1)', '수소이온농도(t-1)', '전기전도도 μS/cm(t-1)',
       '용존산소 mg/L(t-1)', '총유기탄소 mg/L(t-1)', '총질소 mg/L(t-1)', '총인 mg/L(t-1)']

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True)
rnd_reg.fit(test_2015df.loc[:, cols2], test_2015df['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols2):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df.loc[:, cols2], test_2015df['클로로필-a mg/㎥']))

0.15 수온 ℃
0.39 수소이온농도
0.02 전기전도도 μS/cm
0.05 용존산소 mg/L
0.02 총유기탄소 mg/L
0.02 총질소 mg/L
0.01 총인 mg/L
0.18 수온 ℃(t-1)
0.1 수소이온농도(t-1)
0.01 전기전도도 μS/cm(t-1)
0.01 용존산소 mg/L(t-1)
0.01 총유기탄소 mg/L(t-1)
0.02 총질소 mg/L(t-1)
0.01 총인 mg/L(t-1)
oob_scores : 0.8061902230948994
train data 정확도 : 0.8170920185630661


t-2 자료도 추가하면 어떨까?

# 데이터 가져오기

In [ ]:
base_path = '/content/drive/MyDrive/water_data/'

## 승촌보 자료를 데이터프레임으로 만들기  

* `skiprows=0`: 0번 행 무시. 모든 워크시트 대상.
* `header=1`: 1번 행을 header로 지정, 즉 열 인덱스로 사용.
* `sheet_name=None`: 모든 워크시트 가져오기. 워크시트별로 하나의 df 생성. 반환값은 사전.
* `na_values='ND'`: ND(No Data/Not Determinded) 또한 결측치로 처리. 용조산소 열에 포함되어 있음.
* `index_col=0`: 측정일을 행 인덱스로 사용
* `parse_dates=True`: 행 인덱스로 사용되는 날짜 대상 파싱 실행

In [ ]:
scb_yearly = pd.read_excel(base_path+"Seungchonbo.xlsx", 
                            skiprows=0, 
                            header=1, 
                            sheet_name=None,
                            na_values='ND',
                            parse_dates=True)  # index_col = 0을 제외함

In [ ]:
print(type(scb_yearly))  # type-dict
print(scb_yearly.keys())  # 2012년부터 2022년까지

<class 'dict'>
dict_keys(['2012년', '2013년', '2014년', '2015년', '2016년', '2017년', '2018년', '2019년', '2020년', '2021년', '2022년'])


In [ ]:
print(scb_yearly['2014년'].head())

            측정일/월 측정소명  수온 ℃  수소이온농도  전기전도도 μS/cm  용존산소 mg/L  총유기탄소 mg/L  \
0  2014-01-01 00h   용봉   5.8     6.7        412.0        8.4         3.5   
1  2014-01-01 01h   용봉   5.8     6.7        412.0        8.3         3.4   
2  2014-01-01 02h   용봉   5.9     6.7        418.0        8.0         3.4   
3  2014-01-01 03h   용봉   5.9     6.7        421.0        7.8         3.5   
4  2014-01-01 04h   용봉   5.9     6.7        419.0        7.9         3.4   

   총질소 mg/L  총인 mg/L  클로로필-a mg/㎥  
0     6.419    0.058         12.9  
1     6.358    0.059         12.6  
2     6.407    0.059         11.4  
3     6.459    0.059         10.6  
4     6.486    0.059         11.1  


In [ ]:
for year in ['2015년']:
    for var in scb_yearly[year].columns:
        scb_yearly[year][var+'(t-1)'] = 0.0
        scb_yearly[year][var+'(t-2)'] = 0.0
    
    for i in scb_yearly[year].index:
        for var in scb_yearly[year].columns:
            try:
                scb_yearly[year][var+'(t-1)'][i + 1] =  scb_yearly[year][var][i]
            except:
                continue

    for i in scb_yearly[year].index:
        for var in scb_yearly[year].columns:
            try:
                scb_yearly[year][var+'(t-2)'][i + 2] =  scb_yearly[year][var][i]
            except:
                continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


t-1 시간 데이터를 저장함. 만약, t-1 시간 데이터가 결측치라면 0.0을 넣어둠

In [ ]:
print(scb_yearly['2015년'].head())

            측정일/월 측정소명  수온 ℃  수소이온농도  전기전도도 μS/cm  용존산소 mg/L  총유기탄소 mg/L  \
0  2015-01-01 00h   용봉   6.3     7.0        410.0        9.2         2.5   
1  2015-01-01 01h   용봉   6.2     7.0        410.0        9.2         2.5   
2  2015-01-01 02h   용봉   6.1     7.0        409.0        9.2         2.4   
3  2015-01-01 03h   용봉   6.1     7.0        410.0        9.2         2.4   
4  2015-01-01 04h   용봉   6.0     7.0        412.0        9.3         2.4   

   총질소 mg/L  총인 mg/L  클로로필-a mg/㎥  ... 용존산소 mg/L(t-1) 용존산소 mg/L(t-2)  \
0     5.607    0.057          3.9  ...            0.0            0.0   
1     5.596    0.056          3.9  ...            9.2            0.0   
2     5.485    0.053          3.7  ...            9.2            9.2   
3     5.487    0.052          3.7  ...            9.2            9.2   
4     5.625    0.052          3.8  ...            9.2            9.2   

  총유기탄소 mg/L(t-1) 총유기탄소 mg/L(t-2)  총질소 mg/L(t-1)  총질소 mg/L(t-2)  총인 mg/L(t-1)  \
0             0.0            

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
print(scb_yearly['2015년'].isnull().sum())

측정일/월                 0
측정소명                  0
수온 ℃                361
수소이온농도              358
전기전도도 μS/cm         362
용존산소 mg/L           632
총유기탄소 mg/L          771
총질소 mg/L            516
총인 mg/L             615
클로로필-a mg/㎥         323
측정일/월(t-1)            0
측정일/월(t-2)            0
측정소명(t-1)             0
측정소명(t-2)             0
수온 ℃(t-1)           361
수온 ℃(t-2)           361
수소이온농도(t-1)         358
수소이온농도(t-2)         358
전기전도도 μS/cm(t-1)    362
전기전도도 μS/cm(t-2)    362
용존산소 mg/L(t-1)      632
용존산소 mg/L(t-2)      632
총유기탄소 mg/L(t-1)     771
총유기탄소 mg/L(t-2)     771
총질소 mg/L(t-1)       516
총질소 mg/L(t-2)       516
총인 mg/L(t-1)        615
총인 mg/L(t-2)        615
클로로필-a mg/㎥(t-1)    323
클로로필-a mg/㎥(t-2)    323
dtype: int64


전체 결측치는 앞에 있는 값으로 대체함

In [ ]:
test_2015df = scb_yearly['2015년'].fillna(method = 'ffill')

In [ ]:
print(test_2015df.isnull().sum())

측정일/월               0
측정소명                0
수온 ℃                0
수소이온농도              0
전기전도도 μS/cm         0
용존산소 mg/L           0
총유기탄소 mg/L          0
총질소 mg/L            0
총인 mg/L             0
클로로필-a mg/㎥         0
측정일/월(t-1)          0
측정일/월(t-2)          0
측정소명(t-1)           0
측정소명(t-2)           0
수온 ℃(t-1)           0
수온 ℃(t-2)           0
수소이온농도(t-1)         0
수소이온농도(t-2)         0
전기전도도 μS/cm(t-1)    0
전기전도도 μS/cm(t-2)    0
용존산소 mg/L(t-1)      0
용존산소 mg/L(t-2)      0
총유기탄소 mg/L(t-1)     0
총유기탄소 mg/L(t-2)     0
총질소 mg/L(t-1)       0
총질소 mg/L(t-2)       0
총인 mg/L(t-1)        0
총인 mg/L(t-2)        0
클로로필-a mg/㎥(t-1)    0
클로로필-a mg/㎥(t-2)    0
dtype: int64


In [ ]:
test_2015df.columns

Index(['측정일/월', '측정소명', '수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥', '측정일/월(t-1)',
       '측정일/월(t-2)', '측정소명(t-1)', '측정소명(t-2)', '수온 ℃(t-1)', '수온 ℃(t-2)',
       '수소이온농도(t-1)', '수소이온농도(t-2)', '전기전도도 μS/cm(t-1)', '전기전도도 μS/cm(t-2)',
       '용존산소 mg/L(t-1)', '용존산소 mg/L(t-2)', '총유기탄소 mg/L(t-1)',
       '총유기탄소 mg/L(t-2)', '총질소 mg/L(t-1)', '총질소 mg/L(t-2)', '총인 mg/L(t-1)',
       '총인 mg/L(t-2)', '클로로필-a mg/㎥(t-1)', '클로로필-a mg/㎥(t-2)'],
      dtype='object')

t-1 시간 데이터와 함께 랜덤포레스트(회귀)를 돌려봄.  
예상할 수 있듯이.. 클로로필(t-1)만 중요한 변수로 확인됨. 정확도도 높은 편임.

In [ ]:
cols = ['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '수온 ℃(t-1)', '수온 ℃(t-2)',
       '수소이온농도(t-1)', '수소이온농도(t-2)', '전기전도도 μS/cm(t-1)', '전기전도도 μS/cm(t-2)',
       '용존산소 mg/L(t-1)', '용존산소 mg/L(t-2)', '총유기탄소 mg/L(t-1)',
       '총유기탄소 mg/L(t-2)', '총질소 mg/L(t-1)', '총질소 mg/L(t-2)', '총인 mg/L(t-1)',
       '총인 mg/L(t-2)', '클로로필-a mg/㎥(t-1)', '클로로필-a mg/㎥(t-2)']

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True)
rnd_reg.fit(test_2015df.loc[:, cols], test_2015df['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df.loc[:, cols], test_2015df['클로로필-a mg/㎥']))

0.0 수온 ℃
0.0 수소이온농도
0.0 전기전도도 μS/cm
0.0 용존산소 mg/L
0.0 총유기탄소 mg/L
0.0 총질소 mg/L
0.0 총인 mg/L
0.0 수온 ℃(t-1)
0.0 수온 ℃(t-2)
0.0 수소이온농도(t-1)
0.0 수소이온농도(t-2)
0.0 전기전도도 μS/cm(t-1)
0.0 전기전도도 μS/cm(t-2)
0.0 용존산소 mg/L(t-1)
0.0 용존산소 mg/L(t-2)
0.0 총유기탄소 mg/L(t-1)
0.0 총유기탄소 mg/L(t-2)
0.0 총질소 mg/L(t-1)
0.0 총질소 mg/L(t-2)
0.0 총인 mg/L(t-1)
0.0 총인 mg/L(t-2)
1.0 클로로필-a mg/㎥(t-1)
0.0 클로로필-a mg/㎥(t-2)
oob_scores : 0.9775317741328935
train data 정확도 : 0.9799948827179225


클로로필(t-1) 자료를 제외하면.. 클로로필(t-2)가 중요함

In [ ]:
cols2 = ['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '수온 ℃(t-1)', '수온 ℃(t-2)',
       '수소이온농도(t-1)', '수소이온농도(t-2)', '전기전도도 μS/cm(t-1)', '전기전도도 μS/cm(t-2)',
       '용존산소 mg/L(t-1)', '용존산소 mg/L(t-2)', '총유기탄소 mg/L(t-1)',
       '총유기탄소 mg/L(t-2)', '총질소 mg/L(t-1)', '총질소 mg/L(t-2)', '총인 mg/L(t-1)',
       '총인 mg/L(t-2)', '클로로필-a mg/㎥(t-2)']

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True)
rnd_reg.fit(test_2015df.loc[:, cols2], test_2015df['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols2):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df.loc[:, cols2], test_2015df['클로로필-a mg/㎥']))

0.0 수온 ℃
0.0 수소이온농도
0.0 전기전도도 μS/cm
0.0 용존산소 mg/L
0.0 총유기탄소 mg/L
0.0 총질소 mg/L
0.0 총인 mg/L
0.0 수온 ℃(t-1)
0.0 수온 ℃(t-2)
0.0 수소이온농도(t-1)
0.0 수소이온농도(t-2)
0.0 전기전도도 μS/cm(t-1)
0.0 전기전도도 μS/cm(t-2)
0.0 용존산소 mg/L(t-1)
0.0 용존산소 mg/L(t-2)
0.0 총유기탄소 mg/L(t-1)
0.0 총유기탄소 mg/L(t-2)
0.0 총질소 mg/L(t-1)
0.0 총질소 mg/L(t-2)
0.0 총인 mg/L(t-1)
0.0 총인 mg/L(t-2)
0.99 클로로필-a mg/㎥(t-2)
oob_scores : 0.9564180649852755
train data 정확도 : 0.9613159983429178


클로로필(t-1)와 클로로필(t-2) 자료를 제외하면.. 

In [ ]:
cols3 = ['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '수온 ℃(t-1)', '수온 ℃(t-2)',
       '수소이온농도(t-1)', '수소이온농도(t-2)', '전기전도도 μS/cm(t-1)', '전기전도도 μS/cm(t-2)',
       '용존산소 mg/L(t-1)', '용존산소 mg/L(t-2)', '총유기탄소 mg/L(t-1)',
       '총유기탄소 mg/L(t-2)', '총질소 mg/L(t-1)', '총질소 mg/L(t-2)', '총인 mg/L(t-1)',
       '총인 mg/L(t-2)']

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=4, oob_score=True)
rnd_reg.fit(test_2015df.loc[:, cols3], test_2015df['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols3):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df.loc[:, cols3], test_2015df['클로로필-a mg/㎥']))

0.09 수온 ℃
0.39 수소이온농도
0.02 전기전도도 μS/cm
0.05 용존산소 mg/L
0.02 총유기탄소 mg/L
0.02 총질소 mg/L
0.01 총인 mg/L
0.07 수온 ℃(t-1)
0.17 수온 ℃(t-2)
0.07 수소이온농도(t-1)
0.02 수소이온농도(t-2)
0.01 전기전도도 μS/cm(t-1)
0.01 전기전도도 μS/cm(t-2)
0.01 용존산소 mg/L(t-1)
0.0 용존산소 mg/L(t-2)
0.01 총유기탄소 mg/L(t-1)
0.01 총유기탄소 mg/L(t-2)
0.01 총질소 mg/L(t-1)
0.01 총질소 mg/L(t-2)
0.0 총인 mg/L(t-1)
0.0 총인 mg/L(t-2)
oob_scores : 0.8059727088100563
train data 정확도 : 0.8172920212424043


# 계절별로 다르진 않을까? 2015년 데이터 확인

(2015년 기준)  
* 입춘 : 15년 02월 04일
* 입하 : 15년 05월 06일
* 입추 : 15년 08월 08일
* 입동 : 15년 11월 08일

In [ ]:
test_2015df['측정일/월'].tail()

8734    2015-12-31 19h
8735    2015-12-31 20h
8736    2015-12-31 21h
8737    2015-12-31 22h
8738    2015-12-31 23h
Name: 측정일/월, dtype: object

In [ ]:
# 봄   index 816 ~ 2998 
test_2015df_spring = test_2015df[816:2999]
# 여름 index 2999 ~ 5235
test_2015df_summer = test_2015df[2999:5236]
# 가을 index 5236 ~ 7442
test_2015df_fall = test_2015df[2999:5236]
# 겨울 index 0 ~ 815, 7443 ~ 8738
test_2015df_winter = pd.concat([test_2015df[:816], test_2015df[7443:]])

###봄 

수소이온농도(0.49), 수소이온농도(t-1)(0.1), 수온(t-2)(0.08), 용존산소(0.008), 수온(t-1)(0.05), 수온(0.004)

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=4, oob_score=True)
rnd_reg.fit(test_2015df_spring.loc[:, cols3], test_2015df_spring['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols3):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df_spring.loc[:, cols3], test_2015df_spring['클로로필-a mg/㎥']))

0.04 수온 ℃
0.49 수소이온농도
0.02 전기전도도 μS/cm
0.08 용존산소 mg/L
0.03 총유기탄소 mg/L
0.0 총질소 mg/L
0.01 총인 mg/L
0.05 수온 ℃(t-1)
0.08 수온 ℃(t-2)
0.1 수소이온농도(t-1)
0.03 수소이온농도(t-2)
0.01 전기전도도 μS/cm(t-1)
0.01 전기전도도 μS/cm(t-2)
0.01 용존산소 mg/L(t-1)
0.0 용존산소 mg/L(t-2)
0.01 총유기탄소 mg/L(t-1)
0.01 총유기탄소 mg/L(t-2)
0.0 총질소 mg/L(t-1)
0.01 총질소 mg/L(t-2)
0.0 총인 mg/L(t-1)
0.0 총인 mg/L(t-2)
oob_scores : 0.876988724467487
train data 정확도 : 0.8941653873057498


### 여름
수소이온농도(0.27), 용존산소(0.19), 수소이온농도(t-1)(0.17), 전기전도도(01)

여름에는 기본적으로 수온이 높아서 수온이 중요변수로 등장하지 못한 것으로 파악됨.

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=4, oob_score=True)
rnd_reg.fit(test_2015df_summer.loc[:, cols3], test_2015df_summer['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols3):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df_summer.loc[:, cols3], test_2015df_summer['클로로필-a mg/㎥']))

0.0 수온 ℃
0.27 수소이온농도
0.1 전기전도도 μS/cm
0.19 용존산소 mg/L
0.0 총유기탄소 mg/L
0.01 총질소 mg/L
0.02 총인 mg/L
0.0 수온 ℃(t-1)
0.01 수온 ℃(t-2)
0.17 수소이온농도(t-1)
0.03 수소이온농도(t-2)
0.02 전기전도도 μS/cm(t-1)
0.02 전기전도도 μS/cm(t-2)
0.08 용존산소 mg/L(t-1)
0.01 용존산소 mg/L(t-2)
0.0 총유기탄소 mg/L(t-1)
0.0 총유기탄소 mg/L(t-2)
0.0 총질소 mg/L(t-1)
0.01 총질소 mg/L(t-2)
0.01 총인 mg/L(t-1)
0.03 총인 mg/L(t-2)
oob_scores : 0.7315594260133631
train data 정확도 : 0.7755436004439675


### 가을

수소이온농도(0.27), 용존산소(0.19), 수소이온농도(t-1)(0.17), 전기전도도(0.1), 용존산소(t-1)(0.08)

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=4, oob_score=True)
rnd_reg.fit(test_2015df_fall.loc[:, cols3], test_2015df_fall['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols3):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df_fall.loc[:, cols3], test_2015df_fall['클로로필-a mg/㎥']))

0.0 수온 ℃
0.27 수소이온농도
0.1 전기전도도 μS/cm
0.19 용존산소 mg/L
0.0 총유기탄소 mg/L
0.01 총질소 mg/L
0.02 총인 mg/L
0.0 수온 ℃(t-1)
0.01 수온 ℃(t-2)
0.17 수소이온농도(t-1)
0.03 수소이온농도(t-2)
0.02 전기전도도 μS/cm(t-1)
0.02 전기전도도 μS/cm(t-2)
0.08 용존산소 mg/L(t-1)
0.01 용존산소 mg/L(t-2)
0.0 총유기탄소 mg/L(t-1)
0.0 총유기탄소 mg/L(t-2)
0.0 총질소 mg/L(t-1)
0.01 총질소 mg/L(t-2)
0.01 총인 mg/L(t-1)
0.03 총인 mg/L(t-2)
oob_scores : 0.7315594260133631
train data 정확도 : 0.7755436004439675


### 겨울

겨울에는 수온이 중요한 요소로 파악됨.  

수온(t-2)(0.23), 수온(t-1)(0.2), 수온(0.19), 총유기탄소(0.07), 총인(t-2)(0.06)

In [ ]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=4, oob_score=True)
rnd_reg.fit(test_2015df_winter.loc[:, cols3], test_2015df_winter['클로로필-a mg/㎥'])

for score, name in zip(rnd_reg.feature_importances_, cols3):
    print(round(score, 2), name)
print('oob_scores :', rnd_reg.oob_score_)
print('train data 정확도 :', rnd_reg.score(test_2015df_winter.loc[:, cols3], test_2015df_winter['클로로필-a mg/㎥']))

0.19 수온 ℃
0.04 수소이온농도
0.0 전기전도도 μS/cm
0.04 용존산소 mg/L
0.07 총유기탄소 mg/L
0.01 총질소 mg/L
0.01 총인 mg/L
0.2 수온 ℃(t-1)
0.23 수온 ℃(t-2)
0.0 수소이온농도(t-1)
0.0 수소이온농도(t-2)
0.0 전기전도도 μS/cm(t-1)
0.01 전기전도도 μS/cm(t-2)
0.02 용존산소 mg/L(t-1)
0.01 용존산소 mg/L(t-2)
0.04 총유기탄소 mg/L(t-1)
0.02 총유기탄소 mg/L(t-2)
0.01 총질소 mg/L(t-1)
0.03 총질소 mg/L(t-2)
0.01 총인 mg/L(t-1)
0.06 총인 mg/L(t-2)
oob_scores : 0.892002621550293
train data 정확도 : 0.9151079416288022


* 여름과 가을 수온의 평균은 충분히 따뜻함???
* 클로로필은 봄이 제일 많음???


In [ ]:
mean_info = pd.concat([test_2015df_spring.mean(), test_2015df_summer.mean(), test_2015df_fall.mean(), test_2015df_winter.mean()], axis = 1)
mean_info.columns = ['봄', '여름', '가을', '겨울']
mean_info

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,봄,여름,가을,겨울
수온 ℃,12.890151,25.837059,25.837059,9.888400
수소이온농도,7.182272,6.971614,6.971614,6.892140
전기전도도 μS/cm,373.795694,274.421994,274.421994,375.110795
용존산소 mg/L,8.095098,5.228073,5.228073,6.568040
총유기탄소 mg/L,4.000321,4.729459,4.729459,3.699148
총질소 mg/L,7.471354,5.238130,5.238130,5.836849
총인 mg/L,0.104811,0.098116,0.098116,0.127428
클로로필-a mg/㎥,56.029546,32.295574,32.295574,8.399195
수온 ℃(t-1),12.883417,25.832365,25.832365,9.889773
수온 ℃(t-2),12.876683,25.827626,25.827626,9.891098


In [ ]:
median_info = pd.concat([test_2015df_spring.median(), test_2015df_summer.median(), test_2015df_fall.median(), test_2015df_winter.median()], axis = 1)
median_info.columns = ['봄', '여름', '가을', '겨울']
median_info

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/usr/local/lib/p

,봄,여름,가을,겨울
수온 ℃,14.200,25.500,25.500,9.0000
수소이온농도,7.000,6.800,6.800,6.9000
전기전도도 μS/cm,390.000,279.000,279.000,377.0000
용존산소 mg/L,7.800,4.600,4.600,7.1000
총유기탄소 mg/L,3.800,4.700,4.700,3.6000
총질소 mg/L,8.042,5.361,5.361,5.7365
총인 mg/L,0.097,0.091,0.091,0.1110
클로로필-a mg/㎥,45.900,30.400,30.400,6.6000
수온 ℃(t-1),14.200,25.500,25.500,9.0000
수온 ℃(t-2),14.200,25.500,25.500,9.0000


### 에이다부스트 모델

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split

### 봄

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_spring.loc[:, cols3], test_2015df_spring['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols3):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.06 수온 ℃
0.13 수소이온농도
0.01 전기전도도 μS/cm
0.4 용존산소 mg/L
0.02 총유기탄소 mg/L
0.02 총질소 mg/L
0.03 총인 mg/L
0.04 수온 ℃(t-1)
0.06 수온 ℃(t-2)
0.03 수소이온농도(t-1)
0.02 수소이온농도(t-2)
0.01 전기전도도 μS/cm(t-1)
0.03 전기전도도 μS/cm(t-2)
0.02 용존산소 mg/L(t-1)
0.01 용존산소 mg/L(t-2)
0.02 총유기탄소 mg/L(t-1)
0.03 총유기탄소 mg/L(t-2)
0.01 총질소 mg/L(t-1)
0.02 총질소 mg/L(t-2)
0.01 총인 mg/L(t-1)
0.01 총인 mg/L(t-2)
2015년 score : 0.9161632741408139


### 여름

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_summer.loc[:, cols3], test_2015df_summer['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols3):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.02 수온 ℃
0.28 수소이온농도
0.08 전기전도도 μS/cm
0.12 용존산소 mg/L
0.01 총유기탄소 mg/L
0.02 총질소 mg/L
0.05 총인 mg/L
0.01 수온 ℃(t-1)
0.02 수온 ℃(t-2)
0.06 수소이온농도(t-1)
0.1 수소이온농도(t-2)
0.03 전기전도도 μS/cm(t-1)
0.03 전기전도도 μS/cm(t-2)
0.04 용존산소 mg/L(t-1)
0.02 용존산소 mg/L(t-2)
0.01 총유기탄소 mg/L(t-1)
0.01 총유기탄소 mg/L(t-2)
0.01 총질소 mg/L(t-1)
0.02 총질소 mg/L(t-2)
0.02 총인 mg/L(t-1)
0.04 총인 mg/L(t-2)
2015년 score : 0.7491630877789686


### 가을

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_fall.loc[:, cols3], test_2015df_fall['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols3):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.02 수온 ℃
0.28 수소이온농도
0.08 전기전도도 μS/cm
0.12 용존산소 mg/L
0.01 총유기탄소 mg/L
0.02 총질소 mg/L
0.05 총인 mg/L
0.01 수온 ℃(t-1)
0.02 수온 ℃(t-2)
0.06 수소이온농도(t-1)
0.1 수소이온농도(t-2)
0.03 전기전도도 μS/cm(t-1)
0.03 전기전도도 μS/cm(t-2)
0.04 용존산소 mg/L(t-1)
0.02 용존산소 mg/L(t-2)
0.01 총유기탄소 mg/L(t-1)
0.01 총유기탄소 mg/L(t-2)
0.01 총질소 mg/L(t-1)
0.02 총질소 mg/L(t-2)
0.02 총인 mg/L(t-1)
0.04 총인 mg/L(t-2)
2015년 score : 0.7491630877789686


### 겨울

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_winter.loc[:, cols3], test_2015df_winter['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols3):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.27 수온 ℃
0.02 수소이온농도
0.02 전기전도도 μS/cm
0.09 용존산소 mg/L
0.05 총유기탄소 mg/L
0.01 총질소 mg/L
0.01 총인 mg/L
0.16 수온 ℃(t-1)
0.16 수온 ℃(t-2)
0.0 수소이온농도(t-1)
0.0 수소이온농도(t-2)
0.01 전기전도도 μS/cm(t-1)
0.01 전기전도도 μS/cm(t-2)
0.01 용존산소 mg/L(t-1)
0.01 용존산소 mg/L(t-2)
0.04 총유기탄소 mg/L(t-1)
0.08 총유기탄소 mg/L(t-2)
0.01 총질소 mg/L(t-1)
0.03 총질소 mg/L(t-2)
0.01 총인 mg/L(t-1)
0.02 총인 mg/L(t-2)
2015년 score : 0.924454065871908


In [ ]:
cols4 = cols3[:7]
cols4

['수온 ℃',
 '수소이온농도',
 '전기전도도 μS/cm',
 '용존산소 mg/L',
 '총유기탄소 mg/L',
 '총질소 mg/L',
 '총인 mg/L']

### 봄

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_spring.loc[:, cols4], test_2015df_spring['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols4):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.15 수온 ℃
0.2 수소이온농도
0.04 전기전도도 μS/cm
0.44 용존산소 mg/L
0.06 총유기탄소 mg/L
0.08 총질소 mg/L
0.04 총인 mg/L
2015년 score : 0.901859047438738


### 여름

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_summer.loc[:, cols4], test_2015df_summer['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols4):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.05 수온 ℃
0.44 수소이온농도
0.14 전기전도도 μS/cm
0.18 용존산소 mg/L
0.03 총유기탄소 mg/L
0.06 총질소 mg/L
0.1 총인 mg/L
2015년 score : 0.7471248826713268


### 가을

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_fall.loc[:, cols4], test_2015df_fall['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols4):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.05 수온 ℃
0.44 수소이온농도
0.14 전기전도도 μS/cm
0.18 용존산소 mg/L
0.03 총유기탄소 mg/L
0.06 총질소 mg/L
0.1 총인 mg/L
2015년 score : 0.7471248826713268


### 겨울

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(test_2015df_winter.loc[:, cols4], test_2015df_winter['클로로필-a mg/㎥'], random_state=42)

regr = AdaBoostRegressor(RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42, oob_score=True), random_state=42, n_estimators=100)    
regr.fit(X_train, y_train)

for score, name in zip(regr.feature_importances_, cols4):
    print(round(score, 2), name)

print(year, 'score :', regr.score(X_test, y_test))

0.61 수온 ℃
0.03 수소이온농도
0.03 전기전도도 μS/cm
0.07 용존산소 mg/L
0.11 총유기탄소 mg/L
0.1 총질소 mg/L
0.05 총인 mg/L
2015년 score : 0.9038017174959134
